In [250]:
import requests
import pandas as pd

In [251]:
def get_census_data(year, dataset, variables, state=None, county=None, geo="tract", api_key=None):
    """
    Fetch ACS data with flexible geography.
    """
    var_string = ",".join(variables)
    
    base = f"https://api.census.gov/data/{year}/{dataset}"
    url = f"{base}?get={var_string}"
    
    # geography structure
    if geo == "tract":
        url += f"&for=tract:*"
    elif geo == "block group":
        url += f"&for=block%20group:*"
    else:
        url += f"&for={geo}:*"
    
    if state:
        url += f"&in=state:{state}"
    if county:
        url += f"+county:{county}"
    
    if api_key:
        url += f"&key={api_key}"
    
    response = requests.get(url)
    data = response.json()
    
    df = pd.DataFrame(data[1:], columns=data[0])
    return df


In [254]:
df = get_census_data(
    year="2023",
    dataset="acs/acs5",
    variables=[
        "NAME", "B01003_001E",
        "B18101_001E", "B18101_002E", "B18101A_002E", "B18101B_002E",
        "B18101C_002E", "B18101D_002E","B18101I_001E",
        "C18108_003E", "C18108_004E", "C18108_005E", "C18108_006E",
        "C18108_007E", "C18108_008E", 
        "B08201_001E", "B08201_002E", "B08201_003E", "B08201_004E",
        "B08201_005E",
        "B08301_001E", "B08301_002E", "B08301_010E",
        "B08301_018E", "B08301_021E",
        "B01001_020E", "B01001_021E", "B01001_022E",
        "B01001_023E", "B01001_024E", "B01001_025E", "B01001_044E", "B01001_045E", "B01001_046E", "B01001_047E", "B01001_048E","B01001_049E"
    ],
    state="42",
    county="101",
    geo="tract"
)


In [255]:
# Step 1: Rename all variables
rename_dict = {
    "B01003_001E": 'total_population',
    # Disability variables
    'B18101_001E': 'total_population_disability',
    'B18101_002E': 'population_with_disability',
    'B18101A_002E': 'white_population_with_disability',
    'B18101B_002E': 'black_population_with_disability',
    'B18101C_002E': 'american_indian_population_with_disability',
    'B18101D_002E': 'asian_population_with_disability',
    "B18101I_001E": 'hispanic_population_with_disability',
    
    # Disability type variables
    'C18108_003E': 'hearing_difficulty',
    'C18108_004E': 'vision_difficulty',
    'C18108_005E': 'cognitive_difficulty',
    'C18108_006E': 'ambulatory_difficulty',
    'C18108_007E': 'self_care_difficulty',
    'C18108_008E': 'independent_living_difficulty',
    
    # Transportation / Household vehicles
    'B08201_001E': 'total_households',
    'B08201_002E': 'households_no_vehicle',
    'B08201_003E': 'households_1_vehicle',
    'B08201_004E': 'households_2_vehicles',
    'B08201_005E': 'households_3plus_vehicles',
    
    # Commute / Travel
    'B08301_001E': 'total_workers',
    'B08301_002E': 'commute_by_car',
    'B08301_010E': 'commute_by_transit',
    'B08301_018E': 'commute_walk',
    'B08301_021E': 'work_from_home',
    
    # Age (65+) male
    'B01001_020E': 'age_65_66_male',
    'B01001_021E': 'age_67_69_male',
    'B01001_022E': 'age_70_74_male',
    'B01001_023E': 'age_75_79_male',
    'B01001_024E': 'age_80_84_male',
    'B01001_025E': 'age_85_plus_male',

    # Age (65+) female
    "B01001_044E": 'age_65_66_female', 
    "B01001_045E": 'age_67_69_female',
    "B01001_046E": 'age_70_74_female',
    "B01001_047E": 'age_75_79_female',
    "B01001_048E": 'age_80_84_female',
    "B01001_049E": 'age_85_plus_female',
}

# Apply renaming
df = df.rename(columns=rename_dict)

# Step 2: Sum male + female if you added the corresponding female columns
# Example female columns (you would have to rename them similarly)
# 'B01001_044E': 'age_65_66_female', etc.




In [256]:
df.columns

Index(['NAME', 'total_population', 'total_population_disability',
       'population_with_disability', 'white_population_with_disability',
       'black_population_with_disability',
       'american_indian_population_with_disability',
       'asian_population_with_disability',
       'hispanic_population_with_disability', 'hearing_difficulty',
       'vision_difficulty', 'cognitive_difficulty', 'ambulatory_difficulty',
       'self_care_difficulty', 'independent_living_difficulty',
       'total_households', 'households_no_vehicle', 'households_1_vehicle',
       'households_2_vehicles', 'households_3plus_vehicles', 'total_workers',
       'commute_by_car', 'commute_by_transit', 'commute_walk',
       'work_from_home', 'age_65_66_male', 'age_67_69_male', 'age_70_74_male',
       'age_75_79_male', 'age_80_84_male', 'age_85_plus_male',
       'age_65_66_female', 'age_67_69_female', 'age_70_74_female',
       'age_75_79_female', 'age_80_84_female', 'age_85_plus_female', 'state',
       'c

In [257]:
# List of all 65+ male and female columns


count_cols = [
    'total_population',
    'total_population_disability',
    'population_with_disability',
    'white_population_with_disability',
    'black_population_with_disability',
    'american_indian_population_with_disability',
    'asian_population_with_disability',
    'hispanic_population_with_disability',
    'total_households',
    'households_no_vehicle',
    'households_1_vehicle',
    'households_2_vehicles',
    'households_3plus_vehicles',
    'total_workers',
    'commute_by_car',
    'commute_by_transit',
    'commute_walk',
    'work_from_home',
  
]

for col in count_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

cols_65_plus = [
    'age_65_66_male', 'age_67_69_male', 'age_70_74_male',
    'age_75_79_male', 'age_80_84_male', 'age_85_plus_male',
    'age_65_66_female', 'age_67_69_female', 'age_70_74_female',
    'age_75_79_female', 'age_80_84_female', 'age_85_plus_female'
]

# Ensure numeric type
df[cols_65_plus] = df[cols_65_plus].apply(pd.to_numeric, errors='coerce')

# Sum into one column
df['total_age_65_plus'] = df[cols_65_plus].sum(axis=1)

# Optional: compute rate using total population
df['age_65_plus_rate'] = df['total_age_65_plus'] / df['total_population']

In [258]:
# --- 1. Disability rates ---
# total_population_disability is the denominator
disability_cols = [
    'population_with_disability',
    'white_population_with_disability',
    'black_population_with_disability',
    'american_indian_population_with_disability',
    'asian_population_with_disability',
    'hispanic_population_with_disability'
]

for col in disability_cols:
    rate_col = col + "_rate"
    df[rate_col] = df[col] / df['total_population']

# --- 2. Household vehicle rates ---
vehicle_cols = [
    'households_no_vehicle',
    'households_1_vehicle',
    'households_2_vehicles',
    'households_3plus_vehicles'
]

for col in vehicle_cols:
    rate_col = col + "_rate"
    df[rate_col] = df[col] / df['total_households']

df['households_1plus_vehicles']=df['households_1_vehicle']+df['households_2_vehicles']+df['households_3plus_vehicles']
df['households_1plus_vehicles_rate']=df['households_1plus_vehicles']/df['total_households']

# --- 3. Commute method rates ---
commute_cols = [
    'commute_by_car',
    'commute_by_transit',
    'commute_walk',
    'work_from_home'
]

for col in commute_cols:
    rate_col = col + "_rate"
    df[rate_col] = df[col] / df['total_workers']

df['disability_rate'] = df['population_with_disability'] / df['total_population_disability']

In [259]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# =============================
# Step 1: Define mobility-related rates
# =============================

mobility_cols = [
    'disability_rate',       # higher = less mobility
    'age_65_plus_rate',                #higher =  less mobility
    'households_1plus_vehicles_rate',       # higher = better mobility
    'commute_walk_rate',                # higher = better mobility
    'commute_by_transit_rate',     # higher = better mobility
    'commute_by_car_rate',                  #higher= less mobility      
]

# Flip variables where higher is worse for mobility
# For example, population_with_disability_rate, households_no_vehicle_rate
df['disability_rate_inv'] = 1 - df['disability_rate']
df['age_65_plus_rate_inv'] = 1 - df['age_65_plus_rate']
df['commute_by_car_rate_inv'] = 1 - df['commute_by_car_rate']




# Now define final mobility features for the index
mobility_features = [
    'disability_rate_inv',
    'age_65_plus_rate_inv',
    'commute_walk_rate', 
    'commute_by_transit_rate',
    'commute_by_car_rate_inv',
    'households_1plus_vehicles_rate']


# =============================
# Step 2: Normalize features 0-1
# =============================
scaler = MinMaxScaler()

df[mobility_features] = scaler.fit_transform(df[mobility_features])

# =============================
# Step 3: Create mobility index
# =============================
# Simple mean of the normalized features
df['mobility_index'] = df[mobility_features].mean(axis=1)




In [260]:
df.mobility_index

0      0.537189
1      0.465004
2      0.488310
3      0.460903
4      0.496188
         ...   
403         NaN
404         NaN
405    0.597322
406         NaN
407    0.960081
Name: mobility_index, Length: 408, dtype: float64

In [261]:
df_mobility_clean = df.dropna(subset=mobility_features).copy()


In [262]:
min_row = df_mobility_clean.loc[df_mobility_clean['mobility_index'].idxmin()]
max_row = df_mobility_clean.loc[df_mobility_clean['mobility_index'].idxmax()]

print("Min mobility:", min_row['mobility_index'], "→ row:", min_row)
print("Max mobility:", max_row['mobility_index'], "→ row:", max_row)


Min mobility: 0.34177225171739983 → row: NAME                                Census Tract 364; Philadelphia County; Pennsyl...
total_population                                                                 1027
total_population_disability                                                       895
population_with_disability                                                        537
white_population_with_disability                                                   31
                                                          ...                        
disability_rate                                                                   0.6
disability_rate_inv                                                          0.558127
age_65_plus_rate_inv                                                         0.368131
commute_by_car_rate_inv                                                      0.187166
mobility_index                                                               0.341772
Name: 360, Le

In [265]:
df_mobility_clean

,NAME,total_population,total_population_disability,population_with_disability,white_population_with_disability,black_population_with_disability,american_indian_population_with_disability,asian_population_with_disability,hispanic_population_with_disability,hearing_difficulty,...,households_1plus_vehicles_rate,commute_by_car_rate,commute_by_transit_rate,commute_walk_rate,work_from_home_rate,disability_rate,disability_rate_inv,age_65_plus_rate_inv,commute_by_car_rate_inv,mobility_index
0,Census Tract 364; Philadelphia County; Pennsyl...,1027,895,537,31,14,0,0,49,0,...,0.877559,0.757895,0.059649,0.000000,0.182456,0.600000,0.558127,0.368131,0.187166,0.341772
1,Census Tract 356.01; Philadelphia County; Penn...,5279,5122,2262,376,0,0,327,556,25,...,0.643484,0.881798,0.036854,0.000000,0.081348,0.441624,0.779112,0.547737,0.054282,0.343578
2,Census Tract 9802; Philadelphia County; Pennsy...,396,71,32,8,0,0,0,32,0,...,1.000000,0.652174,0.000000,0.000000,0.347826,0.450704,0.766443,0.000000,0.300551,0.344499
3,Census Tract 220; Philadelphia County; Pennsyl...,1459,1354,592,103,1,0,9,118,5,...,0.760445,0.815873,0.085714,0.000000,0.090476,0.437223,0.785253,0.410759,0.124985,0.361193
4,Census Tract 359; Philadelphia County; Pennsyl...,4763,4763,2373,464,0,0,290,50,19,...,0.736672,0.857578,0.012173,0.000000,0.116251,0.498215,0.700149,0.640885,0.080257,0.361689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,Census Tract 28.01; Philadelphia County; Penns...,4175,4175,2024,521,46,0,209,1065,63,...,0.600049,0.310133,0.181818,0.677452,0.201705,0.484790,0.718882,0.876083,0.667386,0.620278
384,Census Tract 87.01; Philadelphia County; Penns...,3831,3831,1356,204,0,0,266,94,0,...,0.462998,0.192551,0.268156,0.357976,0.202607,0.353955,0.901439,0.983661,0.793491,0.627954
385,Census Tract 13.02; Philadelphia County; Penns...,4695,4668,1868,1014,88,0,32,174,14,...,0.699794,0.291814,0.151978,0.581059,0.142186,0.400171,0.836952,0.884910,0.687033,0.640288
386,Census Tract 79; Philadelphia County; Pennsylv...,4120,4120,1643,231,125,0,52,216,0,...,0.699408,0.361989,0.236945,0.670416,0.137123,0.398786,0.838885,0.904504,0.611770,0.660321


In [264]:
# Sort mobility index from lowest to highest
df_mobility_clean = df_mobility_clean.sort_values(by='mobility_index', ascending=True).reset_index(drop=True)
